# Applied Data Science Capstone
## Week 3 -> Segementing and Clustering Neighbourhoods in Toronto

In [2]:
import pandas as pd

### Downloading data from web saving in a dataframe

In [3]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=df[0]
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removing the cells where Borough is not assigned

In [4]:
df=df[~(df['Borough']=='Not assigned')].reset_index(drop=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [5]:
df_cleaned=pd.DataFrame([]) # Dataframe to save post processed data

### Cleaning data to remove the Non assigned Boroughs and adding multiples neighbourhoods in a signle string

In [6]:
df_cleaned['Postcode']=df['Postcode'].unique()
df_cleaned.set_index('Postcode',inplace=True)
df_grouped=df.groupby('Postcode')
for (postcode,group) in df_grouped:
    df_cleaned.loc[postcode,'Borough']=     ', '.join(set(group.Borough))
    df_cleaned.loc[postcode,'Neighborhood']=', '.join(set(group.Neighborhood))
    if df_cleaned.loc[postcode,'Neighborhood']=='Not assigned':
        df_cleaned.loc[postcode,'Neighborhood']=df_cleaned.loc[postcode,'Borough']
df_cleaned.reset_index(drop=False, inplace=True)
df_cleaned.head(5)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [7]:
df_cleaned.shape

(103, 3)